In [8]:
import os
import scanpy as sc
import numpy as np
import muon as mu
np.random.seed(0)
import warnings
warnings.filterwarnings('ignore')

In [9]:
from scipy.stats import median_abs_deviation


def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [10]:
def readData():
    os.chdir("E:/bishe/paper1/data_h5ad/neuips_new")
    rna_file = "RNA_unpred.h5ad"
    rna = sc.read_h5ad(rna_file)
    rna.var_names_make_unique()
    dataset = "neuips"
    atac_file = "ATAC_unpred.h5ad"
    atac = sc.read_h5ad(atac_file)
    atac.var_names_make_unique()
    return rna, atac,dataset

In [11]:
if __name__ == "__main__":

SyntaxError: incomplete input (1147347012.py, line 1)

In [ ]:
    rna, atac, dataname= readData()
    print(dataname)
    rna.var_names_make_unique()
    atac.var_names_make_unique()
    celltype = { "neuips": "cell_type"}
    cell_label = celltype[dataname]

In [ ]:
    rna

In [ ]:
    # mitochondrial genes
    rna.var["mt"] = rna.var_names.str.startswith("MT-")
    # ribosomal genes
    rna.var["ribo"] = rna.var_names.str.startswith(("RPS", "RPL"))
    # hemoglobin genes.
    rna.var["hb"] = rna.var_names.str.contains(("^HB[^(P)]"))
    sc.pp.calculate_qc_metrics(
    rna, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True)
    rna.obs["outlier"] = (
    is_outlier(rna, "log1p_total_counts", 5)
    | is_outlier(rna, "log1p_n_genes_by_counts", 5)
    | is_outlier(rna, "pct_counts_in_top_20_genes", 5))
    rna.obs.outlier.value_counts()
    rna.obs["mt_outlier"] = is_outlier(rna, "pct_counts_mt", 3) | (
    rna.obs["pct_counts_mt"] > 8)
    rna.obs.mt_outlier.value_counts()
    print(f"Total number of cells: {rna.n_obs}")
    adata = rna[(~rna.obs.outlier) & (~rna.obs.mt_outlier)].copy()

    print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")


In [ ]:
    atac = atac[adata.obs_names,:]
    atac

In [12]:
    cell_type_select = ["CD8+ T","Erythroblast"]
    rna_index = []
    for i in range(adata.X.shape[0]):
        if adata.obs[cell_label][i] in cell_type_select:
            rna_index.append(i)
    atac_index = []
    for i in range(atac.X.shape[0]):
        if atac.obs[cell_label][i] in cell_type_select:
            atac_index.append(i)
    
    select_rna = adata[rna_index,:]
    select_atac = atac[atac_index,:]
            

NameError: name 'adata' is not defined

In [ ]:
    select_rna

In [ ]:
    num_cd8 = 490
    num_erythroblast = 10

    # cell_type_select = ["CD4+ T naive" ,"CD4+ T activated","CD8+ T","Erythroblast","Proerythroblast"]
    target_cd8 = select_rna[select_rna.obs[cell_label] == "CD8+ T"].copy()
    target_cd8 = target_cd8[np.random.permutation(len(target_cd8))[:num_cd8]]
    
    target_ery = select_rna[select_rna.obs[cell_label] == "Erythroblast"].copy()
    target_ery = target_ery[np.random.permutation(len(target_ery))[:num_erythroblast]]
    

In [ ]:
    rna_final = ad.concat([target_cd8,target_ery],axis=0)
    rna_final

In [ ]:
    cell_type_counts = rna_final.obs[cell_label].value_counts()

In [ ]:
    atac_final = atac[rna_final.obs_names,:]
    atac_final

In [ ]:
    mdata = mu.MuData({"rna":rna_final,"atac":atac_final})
    mu.write("mdata_rep.h5mu",mdata)